In [1]:
# Importando bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping

In [2]:
# Carregando os dados
df = pd.read_csv('../data/dataset.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df = df[['Close']]
df.head()

,Close
Date,
2018-01-02,23.872063
2018-01-03,23.729017
2018-01-04,23.826284
2018-01-05,24.198215
2018-01-08,24.736080


In [3]:
# Normalizando os dados
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

In [4]:
# Criando os conjuntos de dados de treinamento e teste
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

In [ ]:
# Função para criar os conjuntos de dados de entrada e saída
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

In [ ]:
# Definindo o número de passos de tempo
time_step = 60
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Reshape dos dados para o formato [amostras, passos de tempo, características]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
# Construindo o modelo LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))

In [ ]:
# Compilando o modelo
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Treinando o modelo
early_stopping = EarlyStopping(monitor='loss', patience=5)
model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[early_stopping])

In [ ]:
# Avaliando o modelo
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

In [ ]:
# Invertendo a normalização para calcular o erro
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
# Calculando as métricas de avaliação
train_rmse = np.sqrt(np.mean((train_predict - y_train) ** 2))
test_rmse = np.sqrt(np.mean((test_predict - y_test) ** 2))
print(f'Train RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')

In [ ]:
# Salvando o modelo
model.save('../models/lstm_model.h5')

In [ ]:
# Visualizando os resultados
plt.figure(figsize=(14, 5))
plt.plot(df.index[time_step:len(train_predict) + time_step], train_predict, label='Previsão de Treinamento')
plt.plot(df.index[len(train_predict) + (time_step * 2) + 1:len(df) - 1], test_predict, label='Previsão de Teste')
plt.plot(df['Close'], label='Valor Real')
plt.title('Previsão de Preços de Ações com LSTM')
plt.xlabel('Data')
plt.ylabel('Preço de Fechamento')
plt.legend()
plt.show()